# Mairo - An attempt of teaching Super Mario Bros. to an AI
Yes - it's not a typo. Originally, I just started this project to get into reinforcement learning to use it on Tetris, so this is pretty much just based on tutorials.

[Build an Mario AI Model with Python](https://youtu.be/2eeYqJ0uBKE)

[Playlist: AI development resources](https://www.youtube.com/playlist?list=PLYhl7Si7e0r4RdTPpfKNnzIZpWbLE9NYI)

In [ ]:
%pip install gym-super-mario-bros==7.3.0 nes-py==8.1.0 gym==0.17.3

In [ ]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

# AI Training

In [ ]:
%pip install torch torchvision torchaudio
# stable baselines for reinforcement learning algorithms
%pip install stable-baselines3[extra]

In [ ]:
from gym.wrappers import FrameStack, GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, 'last')

In [ ]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './checkpoints/'
LOG_DIR = './logs/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=1000000, save_path=CHECKPOINT_DIR)

In [ ]:
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

In [ ]:
model.learn(total_timesteps=1000000, callback=callback)

### Saving the model manually

In [ ]:
model.save('./checkpoints/model_26100')

# Viewing the model

In [ ]:
model = PPO.load('./checkpoints/model_26100')
state = env.reset()
while True:
    action, _state = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()